# 1차 전처리 과정을 끝낸 데이터들을 불러오는 과정

## 모델학습용 데이터 병합

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hospital = pd.read_csv('/home/jupyter/climate/data/back_hospital.csv', header = 0)
sfc_prsr = pd.read_csv('/home/jupyter/climate/data/final_sfc_prsr_0723.csv', header = 0)
sfc_td = pd.read_csv('/home/jupyter/climate/data/final_sfc_td_0723.csv', header = 0)
sfc_ta = pd.read_csv('/home/jupyter/climate/data/final_sfc_ta_0723.csv', header = 0)
ydst = pd.read_csv('/home/jupyter/climate/data/final_ydst_pm10_ver1_0723.csv', header = 0)
aws_ta = pd.read_csv('/home/jupyter/climate/data/final_aws_ta_0713.csv', header = 0)
fore_rhm = pd.read_csv('/home/jupyter/climate/data/final_fore_rhm_0801_2313.csv', header = 0)

### 병합의 용이성을 위해 변수명을 바꿔주는 과정

In [ ]:
print(hospital.columns)
print(sfc_prsr.columns)
print(sfc_td.columns)
print(sfc_ta.columns)
print(ydst.columns)
print(aws_ta.columns)
print(fore_rhm.columns)

In [ ]:
hospital = hospital.rename(columns={'back_hospital.area' : 'add'})
hospital = hospital.rename(columns={'back_hospital.yyyymmdd' : 'yyyymmdd'})
hospital = hospital.rename(columns={'back_hospital.sex' : 'sex'})
hospital = hospital.rename(columns={'back_hospital.frequency' : 'frequency'})
ydst = ydst.rename(columns={'area' : 'add'})

### 시도별로 빠진 곳이 있는지 확인하기

In [ ]:
print(hospital['add'].unique())
print(sfc_prsr['add'].unique())
print(sfc_td['add'].unique())
print(sfc_ta['add'].unique())
print(ydst['add'].unique())
print(aws_ta['add'].unique())
print(fore_rhm['add'].unique())

### 파일명에 sfc가 들어있는 데이터들은 세종시 데이터가 없기에 세종시를 둘러싼 각 데이터의 대전, 충남, 충북 평균으로 보간한다.  
### 충남, 충북, 대전에 속한 관측소들의 날짜별 평균을 구하여 각 데이터프레임의 행에 추가하는 코드이다.

In [ ]:
prsr_sejong = sfc_prsr[sfc_prsr['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_prsr = sfc_prsr.append(prsr_sejong)

ta_sejong = sfc_ta[sfc_ta['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_ta = sfc_ta.append(ta_sejong)

td_sejong = sfc_td[sfc_td['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_td = sfc_td.append(td_sejong)

### 각 변수에 따라 시도별 데이터가 필요하기 때문에 각 데이터프레임에서 날짜와 시도가 동일한 행들끼리 평균을 산출하였다. 

In [ ]:
sfc_prsr_grouped = sfc_prsr.groupby(['yyyymmdd','add']).mean()
sfc_ta_grouped = sfc_ta.groupby(['yyyymmdd','add']).mean()
sfc_td_grouped = sfc_td.groupby(['yyyymmdd','add']).mean()
ydst_grouped = ydst.groupby(['yyyymmdd','add']).mean()
aws_ta_grouped = aws_ta.groupby(['yyyymmdd','add']).mean()
fore_rhm_grouped = fore_rhm.groupby(['yyyymmdd','add']).mean()

### 모든 데이터프레임이 동일한 행수를 가지고 있는 것을 확인하고, 모든 데이터프레임을 날짜와 시도에 따라 병합하는 과정을 진행한다. 

In [ ]:
merged_1 = pd.merge(hospital, sfc_prsr_grouped, on = ['add','yyyymmdd'])
merged_2 = pd.merge(merged_1, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_3 = pd.merge(merged_2, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_4 = pd.merge(merged_3, sfc_td_grouped, on = ['add','yyyymmdd'])
merged_5 = pd.merge(merged_4, ydst_grouped, on = ['add','yyyymmdd'])
merged_6 = pd.merge(merged_5, aws_ta_grouped, on = ['add','yyyymmdd'])
merged_7 = pd.merge(merged_6, fore_rhm_grouped, on = ['add','yyyymmdd'])

### 데이터 병합과정에서 불필요하게 추가된 변수들을 제거해주는 코드

In [ ]:
# 모든 변수명 확인
merged_7.columns

In [ ]:
merged_7.drop(['Unnamed: 0_x', 'stn_id_x', 'lat_x', 'long_x', 'isnull', 'stn_id_y', 'lat_y', 'long_y', 'stn_id_x', 'Unnamed: 0_y'], axis=1, inplace=True)

### 데이터 확인 및 저장

In [ ]:
merged_7

In [ ]:
merged_7.to_csv('/home/jupyter/climate/data/final_train.csv')

## 모델평가용 데이터 병합

In [ ]:
hospital = pd.read_csv('/home/jupyter/climate/data/back_testset.csv', header = 0)
sfc_prsr = pd.read_csv('/home/jupyter/climate/data/final_sfc_prsr_0723.csv', header = 0)
sfc_td = pd.read_csv('/home/jupyter/climate/data/final_sfc_td_0723.csv', header = 0)
sfc_ta = pd.read_csv('/home/jupyter/climate/data/final_sfc_ta_0723.csv', header = 0)
ydst = pd.read_csv('/home/jupyter/climate/data/final_ydst_pm10_ver1_0723.csv', header = 0)
aws_ta = pd.read_csv('/home/jupyter/climate/data/final_aws_ta_0713.csv', header = 0)
fore_rhm = pd.read_csv('/home/jupyter/climate/data/final_fore_rhm_0801_2313.csv', header = 0)
hospital = hospital.rename(columns={'back_hospital.area' : 'add'})
hospital = hospital.rename(columns={'back_hospital.yyyymmdd' : 'yyyymmdd'})
hospital = hospital.rename(columns={'back_hospital.sex' : 'sex'})
hospital = hospital.rename(columns={'back_hospital.frequency' : 'frequency'})
ydst = ydst.rename(columns={'area' : 'add'})

hospital = hospital.rename(columns={'area' : 'add'})
ydst = ydst.rename(columns={'area' : 'add'})

In [ ]:
prsr_sejong = sfc_prsr[sfc_prsr['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_prsr = sfc_prsr.append(prsr_sejong)

ta_sejong = sfc_ta[sfc_ta['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_ta = sfc_ta.append(ta_sejong)

td_sejong = sfc_td[sfc_td['add'].isin(['충남', '충북', '대전'])].groupby(['yyyymmdd'], as_index=False).mean().assign(add='세종')
sfc_td = sfc_td.append(td_sejong)

In [ ]:
sfc_prsr_grouped = sfc_prsr.groupby(['yyyymmdd','add']).mean()
sfc_ta_grouped = sfc_ta.groupby(['yyyymmdd','add']).mean()
sfc_td_grouped = sfc_td.groupby(['yyyymmdd','add']).mean()
ydst_grouped = ydst.groupby(['yyyymmdd','add']).mean()
aws_ta_grouped = aws_ta.groupby(['yyyymmdd','add']).mean()
fore_rhm_grouped = fore_rhm.groupby(['yyyymmdd','add']).mean()

### 제공된 테스트 데이터의 날짜 형식 변환

In [ ]:
hospital['yyyymmdd']

In [ ]:
hospital['yyyymmdd'] = pd.to_datetime(hospital['yyyymmdd'], format='%Y-%m-%d')
hospital['yyyymmdd'] = hospital['yyyymmdd'].dt.strftime('%Y%m%d')
hospital['yyyymmdd'] = hospital['yyyymmdd'].astype(int)

In [ ]:
merged_1 = pd.merge(hospital, sfc_prsr_grouped, on = ['add','yyyymmdd'])
merged_2 = pd.merge(merged_1, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_3 = pd.merge(merged_2, sfc_ta_grouped, on = ['add','yyyymmdd'])
merged_4 = pd.merge(merged_3, sfc_td_grouped, on = ['add','yyyymmdd'])
merged_5 = pd.merge(merged_4, ydst_grouped, on = ['add','yyyymmdd'])
merged_6 = pd.merge(merged_5, aws_ta_grouped, on = ['add','yyyymmdd'])
merged_7 = pd.merge(merged_6, fore_rhm_grouped, on = ['add','yyyymmdd'])

In [ ]:
merged_7.columns

In [ ]:
merged_7.drop(['stn_id_x', 'lat_x', 'long_x', 'isnull', 'stn_id_y', 'lat_y', 'long_y', 'Unnamed: 0'], axis=1, inplace=True)

### 데이터 확인 및 저장 저장

In [ ]:
merged_7

In [ ]:
merged_7.to_csv('/home/jupyter/climate/data/final_test.csv')